In [ ]:
# !pip install -q -U google-generativeai

In [1]:
# Import necessary packages and define helper function to render output of models to markdown
import requests
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import Markdown

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY') # Grab API Key from your secrets
genai.configure(api_key=GOOGLE_API_KEY)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Select the model
model = genai.GenerativeModel('gemini-1.5-flash')

### **Example responses for different cases**

The **get_template_response** function gives us simualted LSAS survey responses.

In [ ]:
def get_template_response(level):
  lsas_responses = {
    'mild': ['https://gist.githubusercontent.com/muallagunay/87c0a8724350b14ced9e143fa69261d3/raw/c9975df295e42399c73fd9ae7127c32d98ef8202/LSAS_results_mild.txt'],
    'moderate': ['https://gist.githubusercontent.com/muallagunay/390c572dd08e76f1863475f8ba2b8afb/raw/6893cacc388a254555b260215f28bcfa4721f827/LSAS_results_moderate.txt'],
    'marked': ['https://gist.githubusercontent.com/muallagunay/9981be8eaceeb7e2efd688b4307d44c9/raw/f86a905d8ee63e44908b6cbffa016b7d827a4fa2/LSAS_results_marked.txt'],
    'severe': ['https://gist.githubusercontent.com/muallagunay/bc2f4f7f0145f56789952f0e721cf07f/raw/85f939fc4f4a06f79e053f83dcd4c186bd986f51/LSAS_results_severe.txt'],
    'very_severe': ['https://gist.githubusercontent.com/muallagunay/cb8d1390215fd23400a7ef4d31e2f838/raw/556b5c035b9f3894098fe0f2163b55ec72cb0230/LSAS_results_very_severe.txt']
  }

  link = lsas_responses[level][0]

  # Get the file content
  response = requests.get(link)

  # Check if the request was successful
  if response.status_code == 200:
      return response.text  # The content of the file as a string

## **Prompt Generation**

This **create_prompt** function integrates the client's LSAS responses with additional relevant information for Gemini. The prompt is carefully designed to optimize Gemini's output, ensuring it accurately understands our objectives and provides the most insightful analysis and recommendations.

In [ ]:
def create_prompt(lsas_response):
  aim = "I am a cognitive behavioral therapist specializing in social anxiety in adults, could you please help me:"

  task_one = (
      "Firstly, analyze the Anonymized LSAS Responses, using the client's responses to each of the 24 items on the Liebowitz Social Anxiety Scale (LSAS)."
      "For analyzing, please do the following: a) Identify the specific social situations that elicit the highest levels of fear and avoidance. "
      "b) Determine the core themes and patterns within the client's responses. Here are the LSAS responses: "
      + lsas_response
  )

  task_two = (
      "Secondly, recommend Tailored Interventions. Do this based on the analyzed LSAS responses. Recommending tailored interventions mean"
      "suggesting and providing instructions for "
      "a) one mindfulness exercise the client can use when they feel anxious today (examples include: Daily "
      "Mindfulness practice (focusing on breath, walking), five senses exercise (Notice 5 see, 4 hear, 3 touch, "
      "2 smell, 1 taste while anxious), anchor statements (‘I can handle this moment.’), self-compassion journaling) "
      "b) one cognitive-behavioral exercise the client can try today to reframe their thoughts (examples include: "
      "Thought records, journaling, cognitive restructuring (replacing negative with positive thoughts)) "
      "c) one way the client can incorporate art or crafting into their life to mitigate anxiety today (examples include: "
      "- Art: emotion drawing, social confidence collage (like a manifest board), visual journaling, self-portrait) "
      "- Music: playlist (that helps you calm), improvisation through playing instruments, reflective listening, voice work "
      "- Drama: role rehearsals, character exploration, storytelling, public speaking practice in private "
      "- Movement: movement journaling, empowerment poses, dance your emotions, social flow practice "
      "- Writing: freewriting about emotions, letter writing (letter to your social anxiety as if it were a person), poetic "
      "reflection, affirmation scripts, alternative happy ending, future visualization, letter to future self "
      "- Crafting: mindful crafting, community workshops, gift creation, craft reflection) "
      "d) one resource the client can check out today (e.g., a book, podcast, movie, TV show, paper, thought piece, "
      "article that expresses the voices of others with social anxiety)"
  )

  prompt = aim + task_one + task_two

  return prompt

## **Getting Gemini's response**

We now send the carefully engineered prompt to Gemini and receive its response. Here, we are using the simulated 'mild' case as an example.

In [ ]:
lsas_response = get_template_response('mild') # 'mild', 'moderate', 'marked', 'severe', 'very_severe'
prompt = create_prompt(lsas_response)

# You can now use the model defind about to generate content base on inputs
response = model.generate_content(prompt)

# This makes the response easier to read
to_markdown(response.text)

> ## Analysis of Anonymized LSAS Responses
> 
> **a) Specific Social Situations Eliciting Highest Fear and Avoidance:**
> 
> The client demonstrates moderate levels of social anxiety across various situations, but the highest levels of both fear (score of 2) and avoidance (score of 2) are reported in three specific situations:
> 
> * **Being the center of attention:** This suggests a strong fear of scrutiny and judgment.
> * **Giving a prepared oral talk to a group:**  This points to a significant fear of public speaking and performance anxiety.
> * **Acting, performing, or speaking in front of an audience:** This reinforces the public speaking and performance anxiety theme.
> 
> 
> Other situations causing moderate fear (score of 2) include:
> 
> * Talking to someone in authority.
> * Expressing disagreement or disapproval to someone they don't know well.
> * Trying to make someone's acquaintance for a romantic/sexual relationship.
> * Giving a party (this may involve being the center of attention and managing social interactions).
> 
> Most other situations elicit a fear score of 1, indicating lower levels of anxiety, though avoidance remains present.
> 
> **b) Core Themes and Patterns:**
> 
> The core theme is a fear of negative evaluation and judgment in social situations.  This manifests as avoidance of situations involving public speaking, performance, and being the center of attention.  There's also a pattern of anxiety related to interacting with unfamiliar people or those in positions of authority,  suggesting a possible lack of confidence in social skills and self-efficacy in navigating these interactions. The anxiety seems generalized across various social contexts, though the intensity varies.
> 
> 
> ## Tailored Interventions
> 
> Based on the LSAS analysis, here are some tailored interventions:
> 
> **a) Mindfulness Exercise:** **Five Senses Exercise:**
> 
> * **Instructions:** When feeling anxious today, find a quiet space.  Close your eyes and take a few deep breaths.  Then, systematically notice: 5 things you can see, 4 things you can hear, 3 things you can touch, 2 things you can smell, and 1 thing you can taste.  Focus intently on each sensory detail, allowing yourself to simply observe without judgment. This grounding exercise shifts attention away from anxious thoughts and into the present moment.
> 
> 
> **b) Cognitive-Behavioral Exercise:** **Thought Record:**
> 
> * **Instructions:**  Choose one specific situation from the LSAS that caused anxiety (e.g., speaking up in a meeting).  Use a thought record to identify the situation, the automatic negative thoughts that arose (e.g., "They'll think I'm stupid," "I'll mess up"), the emotions these thoughts caused (e.g., fear, shame), the evidence supporting and contradicting these thoughts, and a more balanced/rational thought (e.g., "I've contributed meaningfully before," "Even if I make a mistake, it's not the end of the world").  This process helps to challenge and reframe negative thought patterns.
> 
> 
> **c) Art/Crafting Intervention:** **Emotion Drawing:**
> 
> * **Instructions:**  Take some time today to express your anxiety through drawing. Don't worry about artistic skill; the goal is to visually represent how your anxiety feels in your body and mind. Use different colors, shapes, and textures to capture the intensity and nuances of the emotion. This can be a powerful way to process feelings and gain a sense of control.
> 
> 
> **d) Resource:** **Podcast Recommendation:**  "Social Anxiety Solutions" podcast by Dr. Thomas A. Richards
> 
> * **Rationale:** This podcast offers interviews with experts in social anxiety and shares strategies for managing the condition.  Hearing from others with social anxiety and learning from their experiences can reduce feelings of isolation and increase hope.
> 
> 
> **Important Note:** These are just initial interventions. A comprehensive treatment plan should involve a regular schedule of sessions incorporating exposure therapy (gradually facing feared situations), cognitive restructuring, social skills training, and relapse prevention strategies.  It's crucial to monitor the client's progress and adjust the treatment plan accordingly.  The interventions above are designed to empower the client to take active steps in managing their anxiety today while setting the stage for more thorough therapeutic work.


#### Marked case example:

In [ ]:
lsas_response = get_template_response('marked') # 'mild', 'moderate', 'marked', 'severe', 'very_severe'
prompt = create_prompt(lsas_response)

# You can now use the model defind about to generate content base on inputs
response = model.generate_content(prompt)

# This makes the response easier to read
to_markdown(response.text)

> ## Analysis of Anonymized LSAS Responses
> 
> **a) Specific Social Situations Eliciting Highest Fear and Avoidance:**
> 
> The client demonstrates the highest levels of fear (score of 3) and avoidance (score of 3) in the following situations:
> 
> * **Acting, performing, or speaking in front of an audience:** This suggests a strong fear of public speaking and performance anxiety.
> * **Being the center of attention:** This indicates a general discomfort with being scrutinized or judged by others.
> * **Giving a prepared oral talk to a group:**  Reinforces the public speaking anxiety.
> * **Trying to make someone's acquaintance for the purpose of a romantic/sexual relationship:** This points to a significant anxiety surrounding initiating romantic relationships.
> * **Giving a party:** This suggests anxiety related to hosting events and managing social interactions within a larger group.
> 
> Situations involving interacting with unfamiliar individuals (talking face-to-face with someone they don't know well, expressing disagreement to someone they don't know well) also elicit moderate levels of fear and avoidance.
> 
> 
> **b) Core Themes and Patterns:**
> 
> The core theme underlying the client's anxiety is a fear of negative evaluation and judgment from others. This manifests in various social situations, particularly those involving public performance, attention, and initiating interactions, especially with strangers or those perceived as authoritative figures.  There's a clear pattern of avoidance behaviors employed to manage the anxiety, ranging from simply avoiding certain social situations (giving a party) to employing smaller avoidance tactics (looking away from someone's eyes).  The anxiety appears to be graded, with more novel and demanding social interactions provoking stronger responses.
> 
> 
> ## Tailored Interventions
> 
> Based on the analysis, here are tailored interventions for the client:
> 
> 
> **a) Mindfulness Exercise:**  **Five Senses Exercise:**
> 
> * **Instructions:** When anxiety arises today,  the client should find a quiet space and focus on grounding themselves using their senses. They should identify: 5 things they can see, 4 things they can hear, 3 things they can touch, 2 things they can smell, and 1 thing they can taste.  This helps shift attention away from anxious thoughts and back to the present moment.  This should be practiced for 2-3 minutes each time.
> 
> 
> **b) Cognitive-Behavioral Exercise:** **Thought Record:**
> 
> * **Instructions:**  The client should keep a thought record throughout the day. Whenever they experience social anxiety, they should write down:
>     * **Situation:**  Describe the specific situation that triggered the anxiety.
>     * **Automatic Thought:** Write down the negative thought or belief that came to mind (e.g., "They'll think I'm stupid," "I'll embarrass myself").
>     * **Emotion:** Describe the emotion they felt (e.g., fear, shame, embarrassment).
>     * **Evidence for the Thought:** List evidence supporting the thought and evidence contradicting the thought.
>     * **Alternative Thought:**  Reframe the negative automatic thought into a more balanced and realistic one (e.g., "It's okay if I make a mistake; everyone does," "I can handle this situation").
>     * **Outcome:** How did the situation end? Did your fears actually come true?
> 
> This exercise helps identify and challenge negative thought patterns.
> 
> 
> **c) Art/Crafting Intervention:** **Emotion Drawing:**
> 
> * **Instructions:** The client should use art materials (crayons, paints, charcoal) to express their feelings of social anxiety. They can choose colors and forms that represent their emotions in the moment. There is no need for it to be a perfect representation, the process of expressing the emotions is what matters.  This can provide a non-verbal outlet for processing difficult emotions.
> 
> 
> **d) Resource:**  **Podcast Recommendation:**  "Social Anxiety Solutions" podcast.
> 
> * **Instructions:**  The client can search for and listen to episodes of the "Social Anxiety Solutions" podcast (or a similar podcast focused on social anxiety). This provides access to expert advice, strategies, and the shared experiences of others dealing with similar challenges, reducing feelings of isolation.
> 
> **Important Note:** These are sample interventions. A therapist should adjust these based on the client's progress and specific needs.  Consistent practice and collaborative refinement of these techniques are crucial for effective treatment of social anxiety.  Regular therapy sessions are essential for providing ongoing support and guidance.


## Very severe case example:

In [ ]:
lsas_response = get_template_response('very_severe') # 'mild', 'moderate', 'marked', 'severe', 'very_severe'
prompt = create_prompt(lsas_response)

# You can now use the model defind about to generate content base on inputs
response = model.generate_content(prompt)

# This makes the response easier to read
to_markdown(response.text)

> ## Analysis of Anonymized LSAS Responses
> 
> **a) Specific Social Situations Eliciting Highest Fear and Avoidance:**
> 
> The client's LSAS responses reveal a consistent level of fear (3) and avoidance (3) across all 24 social situations.  This indicates a pervasive and generalized social anxiety, rather than anxiety specific to a few isolated contexts.  There's no single situation that stands out as significantly more feared or avoided than others. The anxiety appears to be triggered by a broad range of social interactions, encompassing both performance-based situations (public speaking, acting) and everyday interactions (talking to strangers, using a phone in public).
> 
> **b) Core Themes and Patterns:**
> 
> The consistent scoring across all items suggests a core theme of **fear of negative evaluation** and **intense self-consciousness** in social settings. The client seems to anticipate negative judgment from others in almost any social interaction, leading to significant avoidance behaviors. This fear extends to both unfamiliar and relatively familiar social contexts.  The pattern points towards a generalized social anxiety disorder, rather than a specific phobia.
> 
> 
> ## Tailored Interventions
> 
> Based on the pervasive nature of the client's social anxiety, the interventions should focus on building general social confidence and coping skills, rather than targeting specific situations.
> 
> **a) Mindfulness Exercise:**  **Five Senses Exercise:**
> 
> * **Instructions:** When feeling anxious today, find a quiet space.  Close your eyes and systematically focus on each of your senses:
>     * **See:** Notice 5 things you can see. Describe them to yourself internally.  Focus on details – colors, shapes, textures.
>     * **Hear:** Notice 4 things you can hear.  Again, describe them internally.
>     * **Touch:** Notice 3 things you can feel physically.  The texture of your clothes, the temperature of the air, etc.
>     * **Smell:** Notice 2 things you can smell.
>     * **Taste:** Notice 1 thing you can taste (even if it's just your own saliva).
> * **Rationale:** This exercise anchors the client in the present moment, shifting attention away from anxious thoughts about the future or rumination on past social interactions. The focus on sensory details helps to ground them in reality.
> 
> 
> **b) Cognitive-Behavioral Exercise:** **Thought Record:**
> 
> * **Instructions:**  Throughout the day, whenever a situation triggers anxiety, write down:
>     * **Situation:** Describe the situation that triggered anxiety.
>     * **Automatic Thoughts:**  Write down the negative thoughts that occurred (e.g., "They're judging me," "I'm going to make a fool of myself").
>     * **Emotions:** Identify the emotions experienced (e.g., anxiety, fear, shame).
>     * **Evidence for the thought:** List evidence that supports the thought AND evidence that contradicts the thought.
>     * **Alternative Thought:** Reframe the negative thought into a more balanced or realistic one (e.g., "They might not even notice," "Everyone makes mistakes sometimes").
> * **Rationale:** This exercise helps the client identify and challenge their negative automatic thoughts, promoting a more balanced and realistic perspective.  By gathering evidence, they can start to see that their anxious thoughts often lack supporting evidence.
> 
> 
> **c) Art/Crafting Intervention:** **Emotion Drawing:**
> 
> * **Instructions:** When feeling anxious, take out art supplies (pencils, crayons, paints, whatever the client prefers). Don't worry about skill; the goal is expression.  Allow the anxiety to guide your hand.  Draw whatever comes to mind – shapes, colors, lines – that represent how you feel in that moment. Don't try to make it "good," just let it flow.  Afterward, reflect on what the drawing communicates to you.
> * **Rationale:** This non-verbal approach allows the client to externalize their anxiety and process their feelings without the pressure of verbal expression.  It can be a powerful way to understand and manage emotions.
> 
> 
> **d) Resource:** **Podcast: "Social Anxiety Solutions"** (or similar podcast focusing on social anxiety)
> 
> * **Instructions:** Search for podcasts specifically designed for individuals with social anxiety. Many podcasts offer expert advice, coping strategies, and relatable stories from others struggling with similar challenges.
> * **Rationale:**  Hearing the experiences and coping mechanisms of others can be validating and reducing feelings of isolation.  Podcasts can offer practical strategies and support outside of therapy sessions.
> 
> 
> **Important Note:**  These are introductory interventions.  The therapist should monitor the client's progress and adjust the interventions accordingly.  Ongoing therapy is crucial for addressing the underlying cognitive and behavioral patterns maintaining social anxiety.  These exercises are tools to supplement therapy, not replace it.
